<a href="https://colab.research.google.com/github/nosvyat/flask-trading-bot/blob/main/ComfyUI_Flux_1_dev_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

!apt -y install -qq aria2
!pip install -q torchsde

!git clone https://github.com/comfyanonymous/ComfyUI
!git clone https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/ComfyUI-Manager
!git clone https://github.com/ciri/comfyui-model-downloader /content/ComfyUI/custom_nodes/comfyui-model-downloader
%cd /content/ComfyUI

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/ComfyUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/ComfyUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/ComfyUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/ComfyUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64

import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

%cd /content/ComfyUI
!python main.py --dont-print-server

# ⚠ Please click the '.trycloudflare.com' link after seeing 'To view the GUI, go to: http://127.0.0.1:8188'

<>:29: SyntaxWarning: invalid escape sequence '\/'
<>:29: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipython-input-4076010131.py:29: SyntaxWarning: invalid escape sequence '\/'
  tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")


aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
fatal: destination path 'ComfyUI' already exists and is not an empty directory.
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-Manager'...
remote: Enumerating objects: 28007, done.
remote: Counting objects: 100% (622/622), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 28007 (delta 495), reused 335 (delta 323), pack-reused 27385 (from 3)
Receiving objects: 100% (28007/28007), 125.42 MiB | 34.23 MiB/s, done.
Resolving deltas: 100% (20765/20765), done.
Cloning into '/content/ComfyUI/custom_nodes/comfyui-model-downloader'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 195 (delta 9), reused 3 (delta 2), pack-reused 173 (from 1)
Receiving objects: 100% (195/195), 365.18 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/content/ComfyUI
